In [ ]:
import os

import openai
import pandas as pd

In [ ]:
df = pd.read_excel('data/example.xlsx')

In [ ]:
df

In [ ]:
system_prompt = 'Você é um assistente de uma aplicação, no formado de FAQ, e deve retornar as respostas com a resposta apenas se a pergunta tem relação com a aplicação.'

In [ ]:
all_conversations = []

for i, row in df.iterrows():
    all_conversations.append({
        'messages': [
            {
                'role' : 'system',
                'content': system_prompt
            },
            {
                'role' : 'user',
                'content': row['question']
            },
            {
                'role' : 'assistant',
                'content': row['response']
            }
        ]
    })

In [ ]:
all_conversations

In [ ]:
import json

with open('data/instances.jsonl', 'w', encoding='utf-8') as f:
    for conversation in all_conversations:
        json.dump(conversation, f, ensure_ascii=False)
        f.write('\n')

In [ ]:
from dotenv import load_dotenv
load_dotenv()

client = openai.OpenAI(api_key=os.getenv('API_KEY'))

In [ ]:
with open('data/instances.jsonl', 'rb') as f:
    response = client.files.create(file=f, purpose='fine-tune')

In [ ]:
response

In [ ]:
file_id = response.id
file_id

In [ ]:
response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model='gpt-3.5-turbo'
)

In [ ]:
response # wait for finished_at not None

In [ ]:
job_id = response.id
job_id

In [ ]:
client.fine_tuning.jobs.retrieve(job_id) # wait for finished_at not None and fine_tuned_model not None

In [ ]:
fine_tuned_model = client.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
fine_tuned_model